In [1]:
import vectorbt as vbt

In [2]:
import numpy as np
import pandas as pd

## timeseries.py

In [3]:
a = np.random.uniform(size=(1000, 1000))

a.shape

(1000, 1000)

In [6]:
%timeit vbt.timeseries.set_by_mask_nb(a, np.isnan(a), 0)

print(vbt.timeseries.set_by_mask_nb(a, np.isnan(a), 0).shape)

3.6 ms ± 28.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [5]:
%timeit vbt.timeseries.fillna_nb(a, 0)

print(vbt.timeseries.fillna_nb(a, 0).shape)

4.25 ms ± 32.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [6]:
%timeit vbt.timeseries.prepend_nb(a, 1000, np.nan)

print(vbt.timeseries.prepend_nb(a, 1000, np.nan).shape)

5.05 ms ± 85.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(2000, 1000)


In [7]:
%timeit vbt.timeseries.fshift_nb(a, 100)

print(vbt.timeseries.fshift_nb(a, 100).shape)

3.79 ms ± 30.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [8]:
%timeit vbt.timeseries.diff_nb(a, 100)

print(vbt.timeseries.diff_nb(a, 100).shape)

25.8 ms ± 255 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
(1000, 1000)


In [9]:
%timeit vbt.timeseries.pct_change_nb(a)

print(vbt.timeseries.pct_change_nb(a).shape)

11.1 ms ± 84.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [10]:
%timeit vbt.timeseries.ffill_nb(a)

print(vbt.timeseries.ffill_nb(a).shape)

6.67 ms ± 38.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [7]:
%timeit vbt.timeseries.cumsum_nb(a)

print(vbt.timeseries.cumsum_nb(a).shape)

10.1 ms ± 86.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [12]:
%timeit vbt.timeseries.cumprod_nb(a)

print(vbt.timeseries.cumprod_nb(a).shape)

8.57 ms ± 939 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [8]:
%timeit vbt.timeseries.rolling_mean_nb(a, 10)

print(vbt.timeseries.rolling_mean_nb(a, 10).shape)

13.2 ms ± 94.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [9]:
%timeit vbt.timeseries.rolling_std_nb(a, 10)

print(vbt.timeseries.rolling_std_nb(a, 10).shape)

17.5 ms ± 482 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
(1000, 1000)


In [11]:
%timeit vbt.timeseries.expanding_max_nb(a)

print(vbt.timeseries.expanding_max_nb(a).shape)

9.22 ms ± 173 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [12]:
%timeit vbt.timeseries.ewm_mean_nb(a, 10)

print(vbt.timeseries.ewm_mean_nb(a, 10).shape)

12.2 ms ± 52.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [13]:
%timeit vbt.timeseries.ewm_std_nb(a, 10)

print(vbt.timeseries.ewm_std_nb(a, 10).shape)

25 ms ± 4.89 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
(1000, 1000)


## signals.py

In [8]:
%timeit vbt.signals.generate_random_entries_nb((1000, 1000), 100, 2, 42)

entries = vbt.signals.generate_random_entries_nb((1000, 1000), 100, 2, 42)
print(entries.shape)

3.7 ms ± 341 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [9]:
%timeit vbt.signals.generate_random_exits_nb(entries, 42)

exits = vbt.signals.generate_random_exits_nb(entries, 42)
print(exits.shape)

30.2 ms ± 563 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
(1000, 1000)


In [20]:
from numba import njit, b1

@njit
def exit_func_nb(entries, col_idx, prev_idx):
    # First value after entry
    entries = entries[:, col_idx]
    a = np.full(entries.shape, False, dtype=b1)
    a[prev_idx+1:] = True
    return a

In [21]:
%timeit vbt.signals.generate_exits_nb(entries, exit_func_nb, True)

print(vbt.signals.generate_exits_nb(entries, exit_func_nb, True).shape)
# Iterative functions are the most expensive

585 ms ± 2.48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 1000)


In [22]:
@njit
def entry_func_nb(exits, col_idx, prev_idx, a):
    # First value after last entry
    exits = exits[:, col_idx]
    a = a[:, col_idx]
    entries = np.full(exits.shape, False, dtype=b1)
    entries[a > 0.5] = True
    return entries

@njit
def exit_func_nb(entries, col_idx, prev_idx, a):
    # First value after last exit
    entries = entries[:, col_idx]
    a = a[:, col_idx]
    exits = np.full(entries.shape, False, dtype=b1)
    exits[a < 0.5] = True
    return exits

In [23]:
%timeit vbt.signals.generate_entries_and_exits_nb(a.shape, entry_func_nb, exit_func_nb, a)

print(vbt.signals.generate_entries_and_exits_nb(a.shape, entry_func_nb, exit_func_nb, a)[0].shape)
# Iterative functions are the most expensive

2.87 s ± 120 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 1000)


In [24]:
%timeit vbt.signals.rank_true_nb(entries, True)

print(vbt.signals.rank_true_nb(entries, True).shape)

3.5 ms ± 62.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [25]:
%timeit vbt.signals.rank_false_nb(entries, True)

print(vbt.signals.rank_false_nb(entries, True).shape)

5.38 ms ± 39 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [26]:
%timeit vbt.signals.shuffle(entries, 42)

print(vbt.signals.shuffle(entries, 42).shape)

15.5 ms ± 60.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [27]:
%timeit vbt.signals.avg_distance_nb(entries)

print(vbt.signals.avg_distance_nb(entries).shape)

3.73 ms ± 20.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000,)


In [28]:
%timeit vbt.signals.prepend_nb(entries, 100, False)

print(vbt.signals.prepend_nb(entries, 100, False).shape)

1.36 ms ± 24.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
(1100, 1000)


In [29]:
%timeit vbt.signals.fshift_nb(entries, 100)

print(vbt.signals.fshift_nb(entries, 100).shape)

1.37 ms ± 8.81 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
(1000, 1000)


In [30]:
%timeit vbt.signals.first_true_nb(entries, True)

print(vbt.signals.first_true_nb(entries, True).shape)

4.88 ms ± 474 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [31]:
%timeit vbt.signals.first_false_nb(entries, True)

print(vbt.signals.first_false_nb(entries, True).shape)

6.3 ms ± 98.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


## positions.py

In [16]:
%timeit vbt.positions.from_signals_nb(entries, exits)

positions = vbt.positions.from_signals_nb(entries, exits)
print(positions.shape)

4.54 ms ± 250 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


## portfolio.py

In [18]:
%timeit vbt.portfolio.equity_nb(a, positions, 1000, np.full_like(a, 0.1), np.full_like(a, 0.1))

equity = vbt.portfolio.equity_nb(a, positions, 1000, np.full_like(a, 0.1), np.full_like(a, 0.1))
print(equity.shape)

17.3 ms ± 428 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
(1000, 1000)


In [34]:
%timeit vbt.portfolio.trade_profits_nb(equity, positions)

print(vbt.portfolio.trade_profits_nb(equity, positions).shape)

7.53 ms ± 55 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


In [35]:
%timeit vbt.portfolio.trade_returns_nb(equity, positions)

print(vbt.portfolio.trade_returns_nb(equity, positions).shape)

7.83 ms ± 88.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000, 1000)


## strategy.py

In [4]:
%timeit vbt.strategy.dmac_nb(a, np.arange(1, 100), np.arange(1, 100)+1, False, True)

print(vbt.strategy.dmac_nb(a, np.arange(1, 100), np.arange(1, 100)+1, False, True)[0].shape)

2.65 s ± 17.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 99000)


In [5]:
%timeit vbt.strategy.dmac_nb(a, np.arange(1, 100), np.arange(1, 100)+1, True, True)

print(vbt.strategy.dmac_nb(a, np.arange(1, 100), np.arange(1, 100)+1, True, True)[0].shape)
# ewm is much more efficient

2.51 s ± 13.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 99000)


In [6]:
# Test caching
%timeit vbt.strategy.dmac_nb(a, np.full(100, 2), np.full(100, 3), True, True)

print(vbt.strategy.dmac_nb(a, np.full(100, 2), np.full(100, 3), True, True)[0].shape)

1.02 s ± 8.56 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 100000)


In [4]:
%timeit vbt.strategy.bb_nb(a, np.arange(1, 101), np.full(100, 1), False, True)

print(vbt.strategy.bb_nb(a, np.arange(1, 101), np.full(100, 1), False, True)[0].shape)

5.81 s ± 247 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 100000)


In [5]:
%timeit vbt.strategy.bb_nb(a, np.arange(1, 101), np.full(100, 1), True, True)

print(vbt.strategy.bb_nb(a, np.arange(1, 101), np.full(100, 1), True, True)[0].shape)

5.86 s ± 200 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 100000)


In [6]:
# Test caching
%timeit vbt.strategy.bb_nb(a, np.full(100, 2), np.full(100, 1), True, True)

print(vbt.strategy.bb_nb(a, np.full(100, 2), np.full(100, 1), True, True)[0].shape)

2.03 s ± 11.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 100000)


In [4]:
%timeit vbt.strategy.rsi_nb(a, np.arange(1, 100), False, True)

print(vbt.strategy.rsi_nb(a, np.arange(1, 100), False, True).shape)

5.09 s ± 480 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 99000)


In [5]:
%timeit vbt.strategy.rsi_nb(a, np.arange(1, 100), True, True)

print(vbt.strategy.rsi_nb(a, np.arange(1, 100), True, True).shape)

4.6 s ± 360 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 99000)


In [6]:
# Test caching
%timeit vbt.strategy.rsi_nb(a, np.full(100, 2), True, True)

print(vbt.strategy.rsi_nb(a, np.full(100, 2), True, True).shape)

930 ms ± 8.83 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 100000)


In [10]:
%timeit vbt.strategy.stoploss_exit_mask_nb(entries, 0, 0, a.shape[0], a, np.full_like(a, 0.1), True)

print(vbt.strategy.stoploss_exit_mask_nb(entries, 0, 0, a.shape[0], a, np.full_like(a, 0.1), True).shape)

1.56 ms ± 10.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
(1000,)


In [11]:
%timeit vbt.strategy.stoploss_exits_nb(a, entries, np.broadcast_to(0.1, a.shape)[None, :].copy(), True, True)

print(vbt.strategy.stoploss_exits_nb(a, entries, np.broadcast_to(0.1, a.shape)[None, :].copy(), True, True).shape)

844 ms ± 47.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 1000)


In [12]:
stops = np.arange(0, 1, 0.01)
stops = np.tile(stops[:, None][:, None], (1, a.shape[0], a.shape[1]))

print(stops.shape)

(100, 1000, 1000)


In [13]:
%timeit vbt.strategy.stoploss_exits_nb(a, entries, stops, True, True)

print(vbt.strategy.stoploss_exits_nb(a, entries, stops, True, True).shape)

1min 4s ± 154 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 100000)


In [14]:
%timeit vbt.strategy.tstop_exit_mask_nb(entries, 0, 0, a.shape[0], a, np.full_like(a, 0.1), True)

print(vbt.strategy.tstop_exit_mask_nb(entries, 0, 0, a.shape[0], a, np.full_like(a, 0.1), True).shape)

1.74 ms ± 253 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1000,)


In [15]:
%timeit vbt.strategy.tstop_exits_nb(a, entries, np.broadcast_to(0.1, a.shape)[None, :].copy(), True, True)

print(vbt.strategy.tstop_exits_nb(a, entries, np.broadcast_to(0.1, a.shape)[None, :].copy(), True, True).shape)

659 ms ± 5.08 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 1000)


In [16]:
%timeit vbt.strategy.tstop_exits_nb(a, entries, stops, True, True)

print(vbt.strategy.tstop_exits_nb(a, entries, stops, True, True).shape)

59.1 s ± 348 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(1000, 100000)
